In [1]:
import re
import requests as req
from bs4 import BeautifulSoup as bs

import json
import time
from lxml import etree

In [14]:
url = "https://www.applysquare.com/ranking-cn/?country=us"

In [15]:
content = req.get(url)

In [16]:
ins = bs(content.text, 'html.parser')

In [28]:
colleges = ins.find_all(name='div',attrs={"class":"institute-lite "})

In [29]:
a = colleges[0].find(name='a',attrs={"class":"institute-lite-logo"})

In [30]:
a

<a class="institute-lite-logo" href="/institute-cn/princeton/"><img height="60" src="https://cdn.applysquare.net/a2/institute/princeton/logo.png" width="60"/></a>

In [39]:
nation = colleges[0].find(name='span',attrs={"class":""}).text.strip()
nation

'美国'

In [20]:
print(len(colleges))

20


In [25]:
colleges[0].get('href')

'/institute-cn/princeton/'

In [31]:
href = a.get('href')
href

'/institute-cn/princeton/'

In [ ]:
college_page_url = 'https://www.applysquare.com' + href
print(college_page_url)
college_con = req.get(college_page_url)
selector = etree.HTML(college_con.text)

In [74]:
cn = selector.xpath("//a[@class='sp-right']/text()")[0].strip()
print(cn)

普林斯顿大学


In [43]:
en = selector.xpath("//p[@class='ellipsis']/text()")[0].strip()
print(en)

Princeton University


In [94]:
college_json = open('college.json')
college_set = json.load(college_json)

In [80]:
ranks = selector.xpath("//h1[@class='ranking-number pull-left rounded']/text()")
ranks_item = selector.xpath("//h2[@class='ranking-name ellipsis']/text()")

In [81]:
print(ranks)
print(ranks_item)

['1', '7', '11', '6', '17']
['US News', 'TIMES', 'Q.S.', 'ARWU', '美国工程学院']


In [88]:
rank_dict = dict(zip(ranks_item, ranks))

In [89]:
for c in college_set:
    if not c['name'].find(en):
        print(c['name'])
        c['cn'] = cn
        c.update(rank_dict)
        break
    

Princeton University (Princeton)


In [126]:
c_list[0].intersection(c_list[1])

{'university'}

In [ ]:
def similar(s1, s2):
    len(s1.intersection(s2))

In [154]:
def extract_info(c):
    a = c.find(name='a',attrs={"class":"institute-lite-logo"})
    nation = c.find(name='span',attrs={"class":""}).text.strip()
    rank = c.find(name='span',attrs={"class":"institute-lite-ranking"}).text.strip()
    
    href = a.get('href')
    
    college_page_url = 'https://www.applysquare.com' + href
    college_con = req.get(college_page_url)
    selector = etree.HTML(college_con.text)
    cn = selector.xpath("//a[@class='sp-right']/text()")[0].strip()
    en = selector.xpath("//p[@class='ellipsis']/text()")[0].strip().replace(',', ' -')
    print(en)
    ranks = selector.xpath("//h1[@class='ranking-number pull-left rounded']/text()")
    ranks_item = selector.xpath("//h2[@class='ranking-name ellipsis']/text()")
    rank_dict = dict(zip(ranks_item, ranks))
    rank_dict['cn'] = cn
    rank_dict['国家'] = nation
    #enl = en.lower()
    #for c in new_set:
    #    if c['name'].lower().find(enl) > -1:
    #        c.update(rank_dict)       
    #        return False
    rank_dict['name'] = en
    new_set.append(rank_dict)
    print(len(new_set))
    print(college_page_url)
    return True

In [117]:
tl = 'https://www.applysquare.com/ranking-cn/?country=us&page=%d'
extract_info(colleges[10].get('href'))

Virginia Polytechnic Institute and State University
{'name': 'Capitol Technology University (CapTechU)', 'webpage': 'https://www.captechu.edu', 'loc': 'Maryland, United States'}


True

In [105]:
for c in college_set:
    if c['name'].find(u) > -1:
        print(c['name'])
       
        break

In [151]:
len(new_set)

899

In [156]:
tl = 'https://www.applysquare.com/ranking-cn/?page=%d'
import time
for i in range(1, 35):
    url = tl % i
    print(url)
    content = req.get(url)
    ins = bs(content.text, 'html.parser')
    colleges = ins.find_all(name='div',attrs={"class":"institute-lite "})
    
    for c in colleges:
        cn = c.find(name='span',attrs={"class":"institute-school-name"}).text.strip()
        rank = c.find(name='span',attrs={"class":"institute-lite-ranking"}).text.strip()
        #print(cn)
        #print(rank)
        for e in new_set:
            if 'info' in e and 'cn' in e['info'] and e['info']['cn'] == cn:
                break
        else:
            print(cn)
            y = extract_info(c)
        # if y:
            # print('page %d no %d' % (i, colleges.index(c)))
        #time.sleep(0.3)
    # if i % 5 == 0:
college_json = open('college2.json', 'w')
json.dump(new_set, college_json)
college_json.close()

https://www.applysquare.com/ranking-cn/?page=1
洛桑联邦理工学院
Ecole Polytechnique Fédérale de Lausanne
901
https://www.applysquare.com/institute-cn/ch.epfl/
https://www.applysquare.com/ranking-cn/?page=2
加州大学圣地亚哥分校
University of California - San Diego
902
https://www.applysquare.com/institute-cn/ucsd/
https://www.applysquare.com/ranking-cn/?page=3
纽约大学
New York University
903
https://www.applysquare.com/institute-cn/nyu/
慕尼黑工业大学
Technical University of Munich
904
https://www.applysquare.com/institute-cn/de.tum/
https://www.applysquare.com/ranking-cn/?page=4
https://www.applysquare.com/ranking-cn/?page=5
https://www.applysquare.com/ranking-cn/?page=6
https://www.applysquare.com/ranking-cn/?page=7
马里兰大学学院
University of Maryland University College
905
https://www.applysquare.com/institute-cn/umuc/
明尼苏达圣玛丽大学
Saint Mary's University of Minnesota
906
https://www.applysquare.com/institute-cn/smumn/
https://www.applysquare.com/ranking-cn/?page=8
https://www.applysquare.com/ranking-cn/?page=9
米兰理工大学


In [114]:
college_json = open('college2.json', 'w')
json.dump(college_set, college_json)

In [115]:
college_json.close()

In [116]:
len(college_set)

899

In [46]:
l = [c for c in college_set if 'cn' not in c]
len(l)

223

In [104]:
lloc = [c for c in college_set if 'loc' not in c]
len(lloc)

34

In [ ]:
for c in college_set if 'loc' not in c

In [99]:
college2_json = open('college2.json')
college2_set = json.load(college2_json)

In [103]:
for c in lloc:
    keyw = c['name'].lower().replace(' - the state university of new jersey', '')\
            .replace('University', '').replace('-', ' ')\
            .replace(' and ', ' ').replace(' in ',' ').replace(' at ',' ').replace(' of ',' ')\
        .strip()
    print(c['name'] .replace(' - the state university of new jersey', ''))
    ws = keyw.split()
    print(ws)
    for a in college_set:
        if 'loc' not in a:
            continue
        for w in ws:
            if w not in a['name'].lower():
                break
        else:
            print(a['name'])
            a.update(c)
            break
    else:
        print('\t \t \t \t \t' + c['name'])
        continue
    try:
        college_set.remove(c)
    except:
        pass

University of California - Berkeley
['university', 'california', 'berkeley']
University of California – Berkeley (UC Berkeley)
Boston College
['boston', 'college']
	 	 	 	 	Boston College
College of William and Mary
['college', 'william', 'mary']
College of William & Mary (W&M)
University of Wisconsin - Madison
['university', 'wisconsin', 'madison']
University of Wisconsin-Madison (UW Mad)
Penn State University Park
['penn', 'state', 'university', 'park']
	 	 	 	 	Penn State University Park
University of Texas at Austin
['university', 'texas', 'austin']
University of Texas - Austin (UT Austin)
Pepperdine University
['pepperdine', 'university']
	 	 	 	 	Pepperdine University
Yeshiva University
['yeshiva', 'university']
	 	 	 	 	Yeshiva University
University of Minnesota - Twin Cities
['university', 'minnesota', 'twin', 'cities']
University of Minnesota, Twin Cities (UM Twin)
Texas A M University
['texas', 'a', 'm', 'university']
Texas A&M University, College Station (TAMU)
Virginia Poly

In [118]:
new_set = []
for e in college_set:
    #print(e.keys())
    new_item = {'name': e['name'], 'info':{}}
    for k in e:
        if k == 'name':
            continue
        new_item['info'][k] = e[k]
    new_set.append(new_item)
    
len(new_set)

899

In [119]:
for e in new_set:
    print(e['info'].keys())

dict_keys(['webpage', 'loc', 'US News', 'TIMES', 'Q.S.', 'ARWU', '美国工程学院', '美国法学院', '美国商学院', '美国美术学院', 'cn', '国家'])
dict_keys(['webpage', 'loc', 'US News', 'ARWU', '美国工程学院', '美国商学院', 'cn', '国家'])
dict_keys(['webpage', 'loc', 'US News', 'TIMES', 'Q.S.', 'ARWU', '美国工程学院', '美国商学院', 'cn', '国家'])
dict_keys(['webpage', 'loc', 'US News', 'TIMES', 'Q.S.', 'ARWU', '美国工程学院', '美国法学院', '美国商学院', '美国医学院: 研究', 'cn', '国家'])
dict_keys(['webpage', 'loc', 'US News', '美国工程学院', '美国商学院', 'cn', '国家'])
dict_keys(['webpage', 'loc', 'US News', 'Q.S.', 'ARWU', '美国工程学院', '美国商学院', 'cn', '国家'])
dict_keys(['webpage', 'loc', 'US News', '美国工程学院', '美国商学院', 'cn', '国家'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc', 'US News', 'Q.S.', 'ARWU', '美国工程学院', '美国美术学院', 'cn', '国家'])
dict_keys(['webpage', 'loc', 'US News', 'TIMES', 'Q.S.', 'ARWU', '美国工程学院', '美国商学院', 'cn', '国家'])
dict_keys(['webpage', 'loc', 'US News', '美国工程学院', 'cn', '国家'])
dict_keys(['webpage', 'loc', 'US News', 'TIMES', 'Q.S.', 'ARWU', '美国工程学院', '美

dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc', 'US News', '美国文理（Liberal Arts）学院', 'cn', '国家'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc', 'cn', '国家'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['webpage', 'loc'])
dict_keys(['US News', 'TIMES', 'Q.S.', '美国法学院', '美国商学院', 'cn', '国家'])
dict_keys(['US News', 'TIMES', 'Q.S.', 'ARWU', '美国工程学院', '美国商学院', 'cn', '国家'])
dict_keys(['US News', '美国商学院', 'cn', '国家'])
dict_keys(['US News', 'TIMES', 'ARWU', '美国医学院: 临床', '美国医学院: 研究', 'cn', '国家'])
dict_keys(['US News', '美国工程学院', 'cn', '国家'])
dict_keys(['US News', '美国

In [132]:
college_json = open('college4.json', 'w')
json.dump(new_set, college_json)
college_json.close()

In [124]:
for e in new_set:
    if '国家' not in e['info']:
        print( e['name'])
        e['info']['国家'] = '美国'

University of Texas - Arlington (UT Arlingt)
San Jose State University (SJSU)
Pennsylvania State University (Penn State)
Virginia Tech (Virg. Tech)
Rochester Institute of Technology (RIT)
California State University, Fullerton (Fullerton)
Polytechnic Institute Of New York University (NYU Poly)
California State University, Long Beach (CSU LB)
California State University, Los Angeles (CSU LA)
Texas A&M University - Kingsville (TAMUK)
Indiana Univ - Purdue Univ Indianapolis (IUPUI)
Santa Clara University (Santa Clara)
Cleveland State University (Cleveland)
Georgia State University (GSU)
University of Michigan, Dearborn (UM Dearborn)
California State University, East Bay (CSU EB)
Wayne State University (Wayne SU)
University of Houston - Clear Lake (UHCL)
California State University, Fresno (Fresno State)
California State University, Northridge (CSUN)
Northern Illinois University (NIU)
California State University, Sacramento (Sac State)
Wright State University (Wright SU)
Wichita State Univ

In [137]:
nation_list = []
for e in new_set:
    n = e['info']['国家'].split(',')
    if len(n) == 2:
        n = n[1].strip()
    else:
        n = n[0]
    if n in ['香港', '澳门', '台湾']:
        n = '中国' + n
    if n not in nation_list:
        nation_list.append(n)
nation_list

['美国',
 '加拿大',
 '英国',
 '瑞士',
 '新加坡',
 '法国',
 '澳大利亚',
 '中国',
 '中国香港',
 '日本',
 '韩国',
 '荷兰',
 '德国',
 '丹麦',
 '瑞典',
 '比利时',
 '新西兰',
 '阿根廷',
 '芬兰',
 '爱尔兰',
 '俄罗斯',
 '挪威',
 '巴西',
 '马来西亚',
 '智利',
 '以色列',
 '印度',
 '奥地利',
 '西班牙',
 '中国台湾',
 '意大利',
 '沙特阿拉伯',
 '南非',
 '哈萨克斯坦',
 '泰国',
 '墨西哥',
 '捷克',
 '葡萄牙',
 '印度尼西亚',
 '爱沙尼亚',
 '波兰',
 '菲律宾',
 '哥伦比亚',
 '土耳其',
 '阿联酋',
 '匈牙利',
 '埃及',
 '欧盟']